This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

After you register, you will be provided with a unique API key, that you should store:

In [434]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

In [435]:
# First, import the relevant modules
import requests, json, collections


In [436]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=' + API_KEY)

In [437]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
#print(r.json())
column_names = r.json()['dataset']['column_names']
print(column_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### Q1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [438]:
#The entries in 'data' are a list of list, with each list being a day. Need to slice out just the year and match on that to 
#return each entire list.
year_2017 = [x for x in r.json()['dataset']['data'] if x[0][:4] == '2017']
#print(year_2017)
type(year_2017)

list

### Q2. Convert the returned JSON object into a Python dictionary.

In [439]:
#First convert list of lists to list of dicts, and apply column names as well.
list_of_dicts = [dict(zip(column_names, values)) for values in year_2017]
print(type(list_of_dicts))
#print(list_of_dicts)

<class 'list'>


In [440]:
#now take value of first entry and use as key for a nested dictionary, with the remaining values being the inner dictionary.
#pop out the first entry and use as key
AFX_2017 = {}
for item in list_of_dicts:
   name = item.pop('Date')
   AFX_2017[name] = item
print(type(AFX_2017))
#print(AFX_2017)

<class 'dict'>


### Q3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [441]:
open_data = [] # initialze the list
for key in AFX_2017:
    x = AFX_2017[key]['Open']
    if x is not None: # ignore None types
        open_data.append(x) #add values to the list
   
max_val = max(open_data)
min_val = min(open_data)
print(f"Maximum open value is ${max_val}")
print("Minimum open value is $%.2f" % min_val)

Maximum open value is $53.11
Minimum open value is $34.00


### Q4. What was the largest change in any one day (based on High and Low price)?

In [442]:
high_low = []
for key in AFX_2017:
    high = AFX_2017[key]['High']
    low = AFX_2017[key]['Low']
    if [high, low]is not None:
        high_low.append(abs(high - low))
max_high_low = max(high_low)
print("Largest change in any one day was $%.2f" % max_high_low)

Largest change in any one day was $2.81


### Q5. What was the largest change between any two days (based on Closing Price)?

In [443]:
#Assuming this is largest change between two consecutive days...
closing_price = [] #collect the data
for key in AFX_2017:
    close = AFX_2017[key]['Close']
    closing_price.append(close)
#fiunction to find difference between two values
def diff(val1, val2):
    """find the absolute difference between two numbers"""
    val_diff = abs(val2 - val1)
    return(val_diff)
#create list to put pairwise differences in
max_closing = []
x = 0
while x < len(closing_price):
    if x <= 253: #this is messy, I'm sure there is a more elegent way to do this - want to limit value of y to avoid out of range error.
        y = x + 1
    max_closing.append(diff(closing_price[x], closing_price[y])) # apply function to pairwise values
    x += 1
max_value = max(max_closing)     
print("Largest change between any two days, based on Closing Price was $%.2f" % max_value)

Largest change between any two days, based on Closing Price was $2.56


### Q6. What was the average daily trading volume during this year?

In [444]:
traded_volume = []
for key in AFX_2017:
    vol = AFX_2017[key]['Traded Volume']
    if vol is not None:
        traded_volume.append(vol)
ave_vol = sum(traded_volume) / len(traded_volume)
print("Average daily trading volume during 2017 was $%.2f" % ave_vol)

Average daily trading volume during 2017 was $89124.34


### Q7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [445]:
#Median is the middle value for an odd number of items, or the mean of the middle two for an even number.
median = int((len(traded_volume) - 1) / 2) #for a dataset containing uneven number of values, find middle value
sorted_traded_volume = sorted(traded_volume) #for median, list must be sorted!
median_traded_volume = sorted_traded_volume[median]
print("The median trading volume during 2017 was $%.0f" % median_traded_volume)

#test against stats package to ensure correct result
from statistics import median
median(traded_volume)

The median trading volume during 2017 was $76286


76286.0